In [1]:
import pandas as pd
import talib
import os
import logging
import time
import backtrader as bt
import pandas as pd
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo
%matplotlib qt

Loading BokehJS ...

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("study")
handler = logging.FileHandler('study.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [3]:
stock_data_store = pd.HDFStore(r'E:\project\trade_engine\data_processing\quandl_data_processor\stock_day.h5')
symbols_frame = pd.read_csv(r"E:\project\trade_engine\data_processing\quandl_data_processor\stock_symbols.csv")

2021-01-28 00:30:51,991 - numexpr.utils - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-01-28 00:30:51,995 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [162]:
# 取出所有volume>3000000的股票的symbol
symbol_list = pd.read_csv(r"E:\project\trade_engine\data_processing\quandl_data_processor\filter_stock_symbols.csv").loc[:, "symbols"].tolist()

In [163]:
# 一个取出一个symbol
def get_one_symbol_data(symbol):

    symbol_frame = stock_data_store.select('day', where=['symbol=="{}"'.format(symbol)])

    return symbol_frame

In [186]:
# parameters of the indicators
m = 20
n = 40
min_price = 5
float_factor = 0.05
high_factor = 0.3

In [187]:
# a strategy to create buy signal given a symbol and parameters
def get_buy_signal_from_strategy(
    symbol,
    m,
    n,
    min_price,
    float_factor,
    high_factor
):

    data = get_one_symbol_data(symbol)
    data.loc[:, "adj_close_change"] = (data.loc[:, "adj_close"] - data.loc[:, "adj_close"].shift(1)) / data.loc[:, "adj_close"].shift(1)
    data.loc[:, "next_7_days_avg_close_change"] = data["adj_close_change"].rolling(7).mean().shift(-7)
    data.loc[:, "next_14_days_avg_close_change"] = data["adj_close_change"].rolling(14).mean().shift(-14)
    data.loc[:, "next_21_days_avg_close_change"] = data["adj_close_change"].rolling(21).mean().shift(-21)
    data.loc[:, "next_28_days_avg_close_change"] = data["adj_close_change"].rolling(28).mean().shift(-28)
    close_change_m_days_max = data.loc[:, "adj_close"].rolling(m).max()
    close_change_m_days_max_percent = (close_change_m_days_max - data.loc[:, "adj_close"]) / data.loc[:, "adj_close"]

    close_change_m_days_min = data.loc[:, "adj_close"].rolling(m).min()
    close_change_m_days_min_percent = (close_change_m_days_min - data.loc[:, "adj_close"]) / data.loc[:, "adj_close"]

    previous_high = data.loc[:, "adj_close"].rolling(m + n).max()
    previous_high_percent = (previous_high - data.loc[:, "adj_close"]) / data.loc[:, "adj_close"]

    close_greater_than_min_price = (data.loc[:, "adj_close"] > min_price) * 1

    data.loc[:, "buy_signal"] = (
        (close_change_m_days_max_percent < float_factor) 
        & (close_change_m_days_min_percent > -float_factor)
        & (previous_high_percent > high_factor)
        & (close_greater_than_min_price == 1)
    )
    
    return data[data.loc[:, "buy_signal"] == True]

In [190]:
buy_signal_list = []
import time
start_time = time.time()
print("start_time: {}".format(start_time))
for idx, symbol in enumerate(symbol_list):
    buy_signal_data = get_buy_signal_from_strategy(
        symbol,
        m,
        n,
        min_price,
        float_factor,
        high_factor
    )
    buy_signal_list.append(buy_signal_data)
    print("progress{}/{}".format(idx + 1, len(symbol_list)))
end_time = time.time()
print("end_time: {}".format(end_time))

start_time: 1611815783.288158
progress1/1724
progress2/1724
progress3/1724
progress4/1724
progress5/1724
progress6/1724
progress7/1724
progress8/1724
progress9/1724
progress10/1724
progress11/1724
progress12/1724
progress13/1724
progress14/1724
progress15/1724
progress16/1724
progress17/1724
progress18/1724
progress19/1724
progress20/1724
progress21/1724
progress22/1724
progress23/1724
progress24/1724
progress25/1724
progress26/1724
progress27/1724
progress28/1724
progress29/1724
progress30/1724
progress31/1724
progress32/1724
progress33/1724
progress34/1724
progress35/1724
progress36/1724
progress37/1724
progress38/1724
progress39/1724
progress40/1724
progress41/1724
progress42/1724
progress43/1724
progress44/1724
progress45/1724
progress46/1724
progress47/1724
progress48/1724
progress49/1724
progress50/1724
progress51/1724
progress52/1724
progress53/1724
progress54/1724
progress55/1724
progress56/1724
progress57/1724
progress58/1724
progress59/1724
progress60/1724
progress61/1724
pro

progress488/1724
progress489/1724
progress490/1724
progress491/1724
progress492/1724
progress493/1724
progress494/1724
progress495/1724
progress496/1724
progress497/1724
progress498/1724
progress499/1724
progress500/1724
progress501/1724
progress502/1724
progress503/1724
progress504/1724
progress505/1724
progress506/1724
progress507/1724
progress508/1724
progress509/1724
progress510/1724
progress511/1724
progress512/1724
progress513/1724
progress514/1724
progress515/1724
progress516/1724
progress517/1724
progress518/1724
progress519/1724
progress520/1724
progress521/1724
progress522/1724
progress523/1724
progress524/1724
progress525/1724
progress526/1724
progress527/1724
progress528/1724
progress529/1724
progress530/1724
progress531/1724
progress532/1724
progress533/1724
progress534/1724
progress535/1724
progress536/1724
progress537/1724
progress538/1724
progress539/1724
progress540/1724
progress541/1724
progress542/1724
progress543/1724
progress544/1724
progress545/1724
progress546/17

progress971/1724
progress972/1724
progress973/1724
progress974/1724
progress975/1724
progress976/1724
progress977/1724
progress978/1724
progress979/1724
progress980/1724
progress981/1724
progress982/1724
progress983/1724
progress984/1724
progress985/1724
progress986/1724
progress987/1724
progress988/1724
progress989/1724
progress990/1724
progress991/1724
progress992/1724
progress993/1724
progress994/1724
progress995/1724
progress996/1724
progress997/1724
progress998/1724
progress999/1724
progress1000/1724
progress1001/1724
progress1002/1724
progress1003/1724
progress1004/1724
progress1005/1724
progress1006/1724
progress1007/1724
progress1008/1724
progress1009/1724
progress1010/1724
progress1011/1724
progress1012/1724
progress1013/1724
progress1014/1724
progress1015/1724
progress1016/1724
progress1017/1724
progress1018/1724
progress1019/1724
progress1020/1724
progress1021/1724
progress1022/1724
progress1023/1724
progress1024/1724
progress1025/1724
progress1026/1724
progress1027/1724
pro

progress1430/1724
progress1431/1724
progress1432/1724
progress1433/1724
progress1434/1724
progress1435/1724
progress1436/1724
progress1437/1724
progress1438/1724
progress1439/1724
progress1440/1724
progress1441/1724
progress1442/1724
progress1443/1724
progress1444/1724
progress1445/1724
progress1446/1724
progress1447/1724
progress1448/1724
progress1449/1724
progress1450/1724
progress1451/1724
progress1452/1724
progress1453/1724
progress1454/1724
progress1455/1724
progress1456/1724
progress1457/1724
progress1458/1724
progress1459/1724
progress1460/1724
progress1461/1724
progress1462/1724
progress1463/1724
progress1464/1724
progress1465/1724
progress1466/1724
progress1467/1724
progress1468/1724
progress1469/1724
progress1470/1724
progress1471/1724
progress1472/1724
progress1473/1724
progress1474/1724
progress1475/1724
progress1476/1724
progress1477/1724
progress1478/1724
progress1479/1724
progress1480/1724
progress1481/1724
progress1482/1724
progress1483/1724
progress1484/1724
progress14

In [191]:
buy_signal_frame = pd.concat(buy_signal_list)

In [193]:
buy_signal_frame.describe()

,unadj_open,unadj_high,unadj_low,unadj_close,unadj_vol,dividends,splits,adj_open,adj_high,adj_low,adj_close,adj_volume,adj_close_change,next_7_days_avg_close_change,next_14_days_avg_close_change,next_21_days_avg_close_change,next_28_days_avg_close_change
count,8417.000000,8417.000000,8417.000000,8417.000000,8.417000e+03,8417.000000,8417.000000,8.417000e+03,8.417000e+03,8.417000e+03,8.417000e+03,8.417000e+03,8417.000000,8.408000e+03,8399.000000,8395.000000,8.382000e+03
mean,22.463172,22.734243,22.180891,22.455955,2.453251e+06,0.001102,0.999812,2.742201e+07,2.796423e+07,2.694452e+07,2.742928e+07,2.208922e+06,0.000037,1.340073e-03,0.000744,0.000479,3.686423e-04
std,41.309153,41.832398,40.843179,41.420731,6.951381e+06,0.019619,0.012220,1.024078e+09,1.043257e+09,1.005292e+09,1.023128e+09,8.615744e+06,0.015858,3.616528e-02,0.020536,0.016021,1.366307e-02
min,0.030000,0.030000,0.030000,0.030000,0.000000e+00,0.000000,0.166667,4.847876e+00,5.024163e+00,4.800000e+00,5.000467e+00,0.000000e+00,-0.047535,-1.666667e-01,-0.083734,-0.060445,-4.166667e-02
25%,7.110000,7.220000,7.000000,7.140000,9.011500e+04,0.000000,1.000000,9.793175e+00,9.931989e+00,9.690000e+00,9.812312e+00,5.610000e+04,-0.008876,-4.828077e-03,-0.003969,-0.003781,-3.291900e-03
50%,15.650000,15.800000,15.440000,15.650000,5.515000e+05,0.000000,1.000000,1.813928e+01,1.845000e+01,1.795000e+01,1.818737e+01,5.011450e+05,0.000000,2.379049e-17,0.000007,0.000000,2.032105e-17
75%,29.220000,29.565000,28.780000,29.190000,2.131600e+06,0.000000,1.000000,3.617182e+01,3.683000e+01,3.600000e+01,3.623771e+01,1.852077e+06,0.008531,5.252915e-03,0.004251,0.003544,3.030553e-03
max,1960.000000,1992.000000,1932.000000,1965.000000,1.186814e+08,0.650000,1.000000,4.717447e+10,4.717447e+10,4.536007e+10,4.611607e+10,4.463459e+08,0.052448,1.117647e+00,0.558824,0.372549,2.794118e-01


In [194]:
buy_signal_frame

,symbol,unadj_open,unadj_high,unadj_low,unadj_close,unadj_vol,dividends,splits,adj_open,adj_high,adj_low,adj_close,adj_volume,adj_close_change,next_7_days_avg_close_change,next_14_days_avg_close_change,next_21_days_avg_close_change,next_28_days_avg_close_change,buy_signal
date,,,,,,,,,,,,,,,,,,,
2019-06-24,AA,22.53,22.6925,22.160,22.22,2198426.0,0.0,1.0,22.530000,22.692500,22.160000,22.220000,2198426.0,-0.016814,0.003239,0.003179,0.002940,-0.002305,True
2016-06-07,AAL,31.46,32.8700,31.100,32.36,12424206.0,0.0,1.0,30.234532,31.589608,29.888555,31.099474,12424206.0,0.035189,-0.014498,-0.016795,-0.003792,0.004692,True
2018-05-23,AAL,42.64,42.8700,42.270,42.80,3915939.0,0.0,1.0,41.742302,41.967459,41.380091,41.898933,3915939.0,-0.001866,0.006908,0.000353,-0.001517,-0.004341,True
2018-05-30,AAL,43.68,43.8250,42.660,43.30,5457878.0,0.0,1.0,42.760407,42.902354,41.761881,42.388407,5457878.0,-0.006653,-0.000550,-0.002919,-0.005839,-0.003503,True
2008-03-04,AAPL,121.99,124.8800,120.400,124.62,63763700.0,0.0,1.0,15.085427,15.442808,14.888806,15.410656,446345900.0,0.023741,0.004161,0.009224,0.009731,0.006450,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-16,ZYNE,13.30,14.5000,13.180,13.35,120347.0,0.0,1.0,13.300000,14.500000,13.180000,13.350000,120347.0,0.021423,-0.032071,-0.029355,-0.014916,-0.024873,True
2015-12-17,ZYNE,13.55,14.2200,12.550,13.03,82885.0,0.0,1.0,13.550000,14.220000,12.550000,13.030000,82885.0,-0.023970,-0.034301,-0.027479,-0.013825,-0.022203,True
2017-06-07,ZYNE,18.59,19.2500,18.415,18.77,368570.0,0.0,1.0,18.590000,19.250000,18.415000,18.770000,368570.0,0.012952,-0.012825,0.001393,-0.002198,0.001120,True


In [172]:
cerebro = bt.Cerebro()
data = bt.feeds.PandasData(
    dataname=data,
    datetime=None,
    open=8,
    high=9,
    low=10,
    close=11,
    volume=12,
    openinterest=-1
)
cerebro.adddata(data)
cerebro.run()

In [173]:
cerebro.plot(style='candle', iplot=False)

[[<Figure size 640x480 with 4 Axes>]]

In [174]:
# b = Bokeh(style='bar') # 黑底, 单页
b = Bokeh(style='bar', tabs='multi') # 黑底, 多页
# b = Bokeh(style='bar', scheme=Tradimo()) # 传统白底, 单页
# b = Bokeh(style='bar', tabs='multi', scheme=Tradimo()) # 传统白底, 多页
cerebro.plot(b, iplot=False)

[[<backtrader_plotting.bokeh.bokeh.FigurePage at 0x1d79eb8bf60>]]